In [82]:
import pandas as pd
from scipy import stats
import numpy as np

In [83]:
%store -r survey_scores
%store -r shared_control_ssrt
%store -r go_task_accuracy_before_stop_onset
%store -r go_task_accuracy_after_stop_onset
%store -r duration_of_inhibition
%store -r ssrt_first_half
%store -r ssrt_second_half
%store -r simple_stop_ssrt

In [109]:
shared_control_ssrt = shared_control_ssrt.rename_axis('subject_id')
simple_stop_ssrt = simple_stop_ssrt.rename_axis('subject_id')
merged_df = pd.merge(shared_control_ssrt, simple_stop_ssrt[['ssrt', 'ssrt_without_short_ssd_trials', 'ssrt_without_short_ssd_subs']], 
                    left_index=True, right_index=True, how='left')
merged_df.rename(columns={
    'ssrt': 'simple_stop_ssrt',
    'ssrt_without_short_ssd_trials': 'simple_stop_ssrt_without_short_ssd_trials',
    'ssrt_without_short_ssd_subs': 'simple_stop_ssrt_without_short_ssd_subs'
}, inplace=True)

In [85]:
# Remove the last row (the mean row)
merged_df = merged_df.iloc[:-1, :]
%store merged_df
go_task_accuracy_before_stop_onset = go_task_accuracy_before_stop_onset.iloc[:-1, :]
go_task_accuracy_after_stop_onset = go_task_accuracy_after_stop_onset.iloc[:-1, :]
duration_of_inhibition = duration_of_inhibition.iloc[:-1, :]
ssrt_first_half = ssrt_first_half.iloc[:-1, :]
ssrt_second_half = ssrt_second_half.iloc[:-1, :]

Stored 'merged_df' (DataFrame)


## Stats Functions

In [86]:
def cohens_d_paired(x1, x2):
    """Calculate Cohen's d for paired samples"""
    d = (x1 - x2).mean() / np.sqrt(((x1 - x2).std(ddof=1) ** 2) / 2)
    return d

def calculate_ci_for_difference(x1, x2, confidence=0.95):
    """Calculate confidence interval for the mean difference between two paired samples"""
    diff = x1 - x2
    n = len(diff)
    mean_diff = np.mean(diff)
    sem = stats.sem(diff)  # Standard error of the mean
    ci = stats.t.interval(confidence, n-1, loc=mean_diff, scale=sem)
    return mean_diff, ci

In [87]:
def calc_stats_ind(x1, x2):
    """Calculate t-stat, p-value, Cohen's d, DF, 95% CI for two independent samples"""
    t_stat, p_value = stats.ttest_ind(x1, x2, nan_policy='omit')
    print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.4f}")

    mean_x1 = np.mean(x1)
    mean_x2 = np.mean(x2)
    std_x1 = np.std(x1, ddof=1)  # Sample standard deviation
    std_x2 = np.std(x2, ddof=1)

    # Pooled standard deviation
    n1 = len(x1)
    n2 = len(x2)
    pooled_std = np.sqrt(((n1 - 1) * std_x1**2 + (n2 - 1) * std_x2**2) / (n1 + n2 - 2))

    # Cohen's d
    cohens_d = (mean_x1 - mean_x2) / pooled_std
    print(f"Cohen's d: {cohens_d:.3f}")

    # Degrees of freedom
    degrees_of_freedom = n1 + n2 - 2
    print(f"Degrees of Freedom: {degrees_of_freedom}")

    # Confidence Interval Calculation
    mean_diff = mean_x1 - mean_x2
    se_diff = np.sqrt((std_x1**2 / n1) + (std_x2**2 / n2))

    # 95% CI
    ci = stats.t.interval(0.95, df=degrees_of_freedom, loc=mean_diff, scale=se_diff)
    print(f"95% CI for mean difference: [{ci[0]:.3f}, {ci[1]:.3f}]")

## Planned Statistical Tests

### AI-Disengaged vs Non-AI SSRT

In [88]:
# Convert to ms
merged_df.loc[:, 'non_ai'] *= 1000
merged_df.loc[:, 'ai'] *= 1000
merged_df.loc[:, 'ai_ai'] *= 1000

In [89]:
print(f'Mean Non-AI SSRT: {np.mean(merged_df['non_ai'])}')
print(f'Mean AI-Disengaged SSRT: {np.mean(merged_df['ai'])}')
ai_vs_non_ai = stats.ttest_rel(merged_df['ai'], merged_df['non_ai'])
print(f"T-statistic: {ai_vs_non_ai.statistic}, p-value: {ai_vs_non_ai.pvalue}")

Mean Non-AI SSRT: 276.16931123585636
Mean AI-Disengaged SSRT: 298.88444025864635
T-statistic: 5.829069557315188, p-value: 8.947284304246216e-07


In [90]:
cohens_d1 = cohens_d_paired(merged_df['ai'], merged_df['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(merged_df['ai'], merged_df['non_ai'])
print(f"Cohen's d: {cohens_d1:.3f}")
print(f"Mean difference = {mean_diff1:.3f} ms")
print(f"95% CI: [{ci1[0]:.3f}, {ci1[1]:.3f}] ms")

Cohen's d: 1.303
Mean difference = 22.715 ms
95% CI: [14.833, 30.597] ms


### AI-Engaged vs AI-Disengaged SSRT

In [91]:
print(f'Mean AI-Engaged SSRT: {np.mean(merged_df['ai_ai'])}')
print(f'Mean AI-Disengaged SSRT: {np.mean(merged_df['ai'])}')
ai_ai_vs_ai = stats.ttest_rel(merged_df['ai_ai'], merged_df['ai'])
print(f"T-statistic: {ai_ai_vs_ai.statistic}, p-value: {ai_ai_vs_ai.pvalue}")

Mean AI-Engaged SSRT: 331.0309391556003
Mean AI-Disengaged SSRT: 298.88444025864635
T-statistic: 3.6632559828548246, p-value: 0.0007385395772012454


In [92]:
cohens_d1 = cohens_d_paired(merged_df['ai_ai'], merged_df['ai'])
mean_diff1, ci1 = calculate_ci_for_difference(merged_df['ai_ai'], merged_df['ai'])
print(f"Cohen's d: {cohens_d1:.3f}")
print(f"Mean difference = {mean_diff1:.3f} ms")
print(f"95% CI: [{ci1[0]:.3f}, {ci1[1]:.3f}] ms")

Cohen's d: 0.819
Mean difference = 32.146 ms
95% CI: [14.397, 49.896] ms


### Non-AI SSRT vs Simple Stop SSRT

In [93]:
print(f'Mean Non-AI SSRT: {np.mean(merged_df['non_ai'])}')
print(f'Simple Stop SSRT: {np.mean(merged_df['simple_stop_ssrt'])}')
non_ai_vs_simple = stats.ttest_rel(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
non_ai_vs_simple_corr = np.corrcoef(merged_df['non_ai'], merged_df['simple_stop_ssrt'])[1][0]
print(f"T-statistic: {non_ai_vs_simple.statistic}, p-value = {non_ai_vs_simple.pvalue}")
print(f"Correlation: {non_ai_vs_simple_corr}")

Mean Non-AI SSRT: 276.16931123585636
Simple Stop SSRT: 213.1577723907928
T-statistic: 13.16166244544354, p-value = 6.286586264119194e-16
Correlation: 0.4732056062671472


In [94]:
cohens_d2 = cohens_d_paired(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
mean_diff2, ci2 = calculate_ci_for_difference(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
print(f"Cohen's d: {cohens_d2:.3f}")
print(f"Mean difference = {mean_diff2:.3f} ms")
print(f"95% CI: [{ci2[0]:.3f}, {ci2[1]:.3f}] ms")

Cohen's d: 2.943
Mean difference = 63.012 ms
95% CI: [53.328, 72.695] ms


## Order Effects

### Calculate order effects between subjects who had the AI block first vs. the Non-AI block first

In [95]:
# Initialize subjects who had the Non-AI and AI Blocks first
non_ai_first_subs = ['s004', 's009', 's008', 's011', 's012', 's015', 's016', 's019', 's020', 's023', 's024', 's027', 's028', 's031', 's032', 's035', 's036', 's039', 's040', 's043']
ai_first_subs = ['s005', 's006', 's007', 's010', 's013', 's014', 's017', 's018', 's021', 's022', 's025', 's026', 's029', 's030', 's033', 's034', 's037', 's038', 's041', 's042']

In [96]:
ssrt_non_ai_first_non_ai = []
ssrt_ai_first_non_ai = []
ssrt_non_ai_first_ai = []
ssrt_ai_first_ai = []
ssrt_non_ai_first_ai_ai = []
ssrt_ai_first_ai_ai = []

for sub in non_ai_first_subs:
    ssrt_non_ai_first_non_ai.append(shared_control_ssrt.loc[sub, "non_ai"])
    ssrt_non_ai_first_ai.append(shared_control_ssrt.loc[sub, "ai"])
    ssrt_non_ai_first_ai_ai.append(shared_control_ssrt.loc[sub, "ai_ai"])

for sub in ai_first_subs:
    ssrt_ai_first_non_ai.append(shared_control_ssrt.loc[sub, "non_ai"])
    ssrt_ai_first_ai.append(shared_control_ssrt.loc[sub, "ai"])
    ssrt_ai_first_ai_ai.append(shared_control_ssrt.loc[sub, "ai_ai"])

# Calculate the differences between AI SSRT and Non-AI SSRT for both subject groups
diff_non_ai_first = np.array(ssrt_non_ai_first_ai) - np.array(ssrt_non_ai_first_non_ai)  # AI - Non-AI for Non-AI first subjects
diff_ai_first = np.array(ssrt_ai_first_ai) - np.array(ssrt_ai_first_non_ai)  # AI - Non-AI for AI first subjects

calc_stats_ind(diff_ai_first, diff_non_ai_first)
mean_ai_first = diff_ai_first.mean()
mean_non_ai_first = diff_non_ai_first.mean()

ci_ai_first = stats.t.interval(.95, len(diff_ai_first)-1, loc=mean_ai_first, scale=stats.sem(diff_ai_first))
print(f"Mean difference (AI first): {mean_ai_first:.2f} ms, 95% CI: [{ci_ai_first[0]:.2f}, {ci_ai_first[1]:.2f}] ms")

ci_non_ai_first = stats.t.interval(.95, len(diff_non_ai_first)-1, loc=mean_non_ai_first, scale=stats.sem(diff_non_ai_first))
print(f"Mean difference (Non-AI first): {mean_non_ai_first:.2f} ms, 95% CI: [{ci_non_ai_first[0]:.2f}, {ci_non_ai_first[1]:.2f}] ms")


T-statistic: 1.04, P-value: 0.3071
Cohen's d: 0.327
Degrees of Freedom: 38
95% CI for mean difference: [-0.008, 0.024]
Mean difference (AI first): 0.03 ms, 95% CI: [0.01, 0.04] ms
Mean difference (Non-AI first): 0.02 ms, 95% CI: [0.01, 0.03] ms


### Calculate AI-Disengaged vs. Non-AI SSRT depending on whether they occurred in the first half or second half of the block

#### First half

In [97]:
# T test for AI vs Non AI SSRT in the first half of trials
ssrt_first_half.loc[:, 'ai_ssrt_first_half'] *= 1000
ssrt_first_half.loc[:, 'non_ai_ssrt_first_half'] *= 1000

ai_vs_non_ai = stats.ttest_rel(ssrt_first_half['ai_ssrt_first_half'], ssrt_first_half['non_ai_ssrt_first_half'])
print(f"T-statistic: {ai_vs_non_ai.statistic:.3f}, p-value: {ai_vs_non_ai.pvalue:.4f}")

cohens_d1 = cohens_d_paired(ssrt_first_half['ai_ssrt_first_half'], ssrt_first_half['non_ai_ssrt_first_half'])
mean_diff1, ci1 = calculate_ci_for_difference(ssrt_first_half['ai_ssrt_first_half'], ssrt_first_half['non_ai_ssrt_first_half'])
print(f"Cohen's d: {cohens_d1:.3f}")
print(f"Mean difference: {mean_diff1:.3f} ms")
print(f"95% CI: [{ci1[0]:.3f}, {ci1[1]:.3f}] ms")

print(f'Mean Non-AI SSRT First Half: {np.mean(ssrt_first_half['non_ai_ssrt_first_half'])}')
print(f'Mean AI-Disengaged SSRT First Half: {np.mean(ssrt_first_half['ai_ssrt_first_half'])}')

T-statistic: 3.283, p-value: 0.0022
Cohen's d: 0.734
Mean difference: 21405489301.756 ms
95% CI: [8217011488.102, 34593967115.410] ms
Mean Non-AI SSRT First Half: 273863893182.94824
Mean AI-Disengaged SSRT First Half: 295269382484.7041


#### Second Half

In [98]:
ssrt_second_half.loc[:, 'ai_ssrt_second_half'] *= 1000
ssrt_second_half.loc[:, 'non_ai_ssrt_second_half'] *=1000
ai_vs_non_ai = stats.ttest_rel(ssrt_second_half['ai_ssrt_second_half'], ssrt_second_half['non_ai_ssrt_second_half'])
print(f"T-statistic: {ai_vs_non_ai.statistic:.3f}, p-value: {ai_vs_non_ai.pvalue:.4f}")

cohens_d1 = cohens_d_paired(ssrt_second_half['ai_ssrt_second_half'], ssrt_second_half['non_ai_ssrt_second_half'])
mean_diff1, ci1 = calculate_ci_for_difference(ssrt_second_half['ai_ssrt_second_half'], ssrt_second_half['non_ai_ssrt_second_half'])
print(f"Cohen's d: {cohens_d1:.3f}")
print(f"Mean difference = {mean_diff1:.3f} ms")
print(f"95% CI: [{ci1[0]:.3f}, {ci1[1]:.3f}] ms")

print(f'Mean Non-AI SSRT Second Half: {np.mean(ssrt_second_half['non_ai_ssrt_second_half'])}')
print(f'Mean AI-Disengaged SSRT Second Half: {np.mean(ssrt_second_half['ai_ssrt_second_half'])}')

T-statistic: 4.820, p-value: 0.0000
Cohen's d: 1.078
Mean difference = 25485122537.610 ms
95% CI: [14790540371.065, 36179704704.156] ms
Mean Non-AI SSRT Second Half: 278474435534.9745
Mean AI-Disengaged SSRT Second Half: 303959558072.58466


## Exploratory T-tests and Confidence Intervals

### Non-AI vs AI-Disengaged Duration of Inhibition

In [99]:
duration_of_inhibition.loc[:, 'ai'] *= 1000
duration_of_inhibition.loc[:, 'non_ai'] *= 1000
t_stat, p_value = stats.ttest_rel(duration_of_inhibition['ai'], duration_of_inhibition['non_ai'])

print(f"T-statistic: {t_stat:.3f}, P-value: {p_value:.4f}")

cohens_d1 = cohens_d_paired(duration_of_inhibition['ai'], 
                            duration_of_inhibition['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(duration_of_inhibition['ai'], 
                                              duration_of_inhibition['non_ai'])
print(f"Cohen's d: {cohens_d1:.3f}")
print(f"Mean difference: {mean_diff1:.3f} ms")
print(f"95% CI: [{ci1[0]:.3f}, {ci1[1]:.3f}] ms")

print(f'Mean Non-AI Duration of Inhibition: {np.mean(duration_of_inhibition['non_ai'])}')
print(f'Mean AI-Disengaged Duration of Inhibition: {np.mean(duration_of_inhibition['ai'])}')

T-statistic: 2.692, P-value: 0.0104
Cohen's d: 0.602
Mean difference: 5763750019.577 ms
95% CI: [1432575029.256, 10094925009.898] ms
Mean Non-AI Duration of Inhibition: 41473963705.935715
Mean AI-Disengaged Duration of Inhibition: 47237713725.513054


### Non-AI vs AI-Disengaged Go Task Accuracy Before Stop Onset

In [100]:
mean_non_ai = go_task_accuracy_before_stop_onset['non_ai'].mean()
mean_ai_disengaged = go_task_accuracy_before_stop_onset['ai'].mean()

print(f"Mean Non-AI Accuracy: {mean_non_ai:.2f}")
print(f"Mean AI-Disengaged Accuracy: {mean_ai_disengaged:.2f}")

t_stat, p_value = stats.ttest_rel(go_task_accuracy_before_stop_onset['ai'], go_task_accuracy_before_stop_onset['non_ai'])

print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")

cohens_d1 = cohens_d_paired(go_task_accuracy_before_stop_onset['ai'], go_task_accuracy_before_stop_onset['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(go_task_accuracy_before_stop_onset['ai'], go_task_accuracy_before_stop_onset['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference = {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")


Mean Non-AI Accuracy: 0.87
Mean AI-Disengaged Accuracy: 0.90
T-statistic: 2.81, P-value: 0.008
Cohen's d: 0.63
Mean difference = 0.03 ms
95% CI: [0.01, 0.06] ms


### Non-AI vs AI-Disengaged Go Task Accuracy After Stop Onset (aka Stop Success Rate)

In [101]:
mean_non_ai = go_task_accuracy_after_stop_onset['non_ai'].mean()
mean_ai_disengaged = go_task_accuracy_after_stop_onset['ai'].mean()

print(f"Mean Non-AI Accuracy: {mean_non_ai:.2f}")
print(f"Mean AI Disengaged Accuracy: {mean_ai_disengaged:.2f}")

t_stat, p_value = stats.ttest_rel(go_task_accuracy_after_stop_onset['ai'], go_task_accuracy_after_stop_onset['non_ai'])

print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")

cohens_d1 = cohens_d_paired(go_task_accuracy_after_stop_onset['ai'], go_task_accuracy_after_stop_onset['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(go_task_accuracy_after_stop_onset['ai'], go_task_accuracy_after_stop_onset['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference = {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")

Mean Non-AI Accuracy: 0.59
Mean AI Disengaged Accuracy: 0.40
T-statistic: -8.88, P-value: 0.000
Cohen's d: -1.98
Mean difference = -0.19 ms
95% CI: [-0.24, -0.15] ms


### Non-AI vs AI-Disengaged SSRT in subjects who did not show proactive slowing

In [102]:
# Identify non-proactive slowing subjects
non_proactive_slowing_mask = (go_task_accuracy_before_stop_onset['non_ai'] > 
                               go_task_accuracy_before_stop_onset['ai'])
non_proactive_slowing_subs = go_task_accuracy_before_stop_onset.index[non_proactive_slowing_mask].tolist()
ssrt_non_ai = shared_control_ssrt['non_ai']
ssrt_ai = shared_control_ssrt['ai']

ssrt_data = pd.DataFrame({
    'subject_id': shared_control_ssrt.index,
    'ssrt_non_ai': ssrt_non_ai,
    'ssrt_ai': ssrt_ai
})

ssrt_data = ssrt_data[ssrt_data['subject_id'] != 'mean'] #remove mean row

ssrt_data['difference_ms'] = (ssrt_data['ssrt_ai'] - ssrt_data['ssrt_non_ai'])*1000 # find ai-disengaged - non ai ssrt
ssrt_data['slowing_type'] = ['Non-Proactive' if subject in non_proactive_slowing_subs else 'Proactive' 
                              for subject in ssrt_data['subject_id']]

non_proactive_slowing_data = ssrt_data[ssrt_data['slowing_type'] == 'Non-Proactive']

In [103]:
non_proactive_slowing_data.loc[:, 'ssrt_ai'] *= 1000
non_proactive_slowing_data.loc[:, 'ssrt_non_ai'] *= 1000
t_stat, p_value = stats.ttest_rel(non_proactive_slowing_data['ssrt_ai'], non_proactive_slowing_data['ssrt_non_ai'])

print(f"T-statistic: {t_stat:.3f}, P-value: {p_value:.4f}")

cohens_d1 = cohens_d_paired(non_proactive_slowing_data['ssrt_ai'], non_proactive_slowing_data['ssrt_non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(non_proactive_slowing_data['ssrt_ai'], non_proactive_slowing_data['ssrt_non_ai'])
print(f"Cohen's d: {cohens_d1:.3f}")
print(f"Mean difference = {mean_diff1:.3f} ms")
print(f"95% CI: [{ci1[0]:.3f}, {ci1[1]:.3f}] ms")

print(f'Mean Non-AI SSRT: {np.mean(non_proactive_slowing_data['ssrt_non_ai'])}')
print(f'Mean AI-Disengaged SSRT: {np.mean(non_proactive_slowing_data['ssrt_ai'])}')

T-statistic: 2.936, P-value: 0.0166
Cohen's d: 1.313
Mean difference = 30.018 ms
95% CI: [6.893, 53.142] ms
Mean Non-AI SSRT: 267.6881218329072
Mean AI-Disengaged SSRT: 297.70585892177377


### T-test comparing AI-Disengaged minus Non-AI SSRT in subjects who proactively slowed and did not proactively slow

In [104]:
non_proactive_differences = ssrt_data[ssrt_data['slowing_type'] == 'Non-Proactive']['difference_ms'].dropna()
proactive_differences = ssrt_data[ssrt_data['slowing_type'] == 'Proactive']['difference_ms'].dropna()

calc_stats_ind(non_proactive_differences, proactive_differences)
mean_non_proactive = ssrt_data[ssrt_data['slowing_type'] == 'Non-Proactive']['difference_ms'].mean()
mean_proactive = ssrt_data[ssrt_data['slowing_type'] == 'Proactive']['difference_ms'].mean()

print(f"Mean difference (Non-Proactive): {mean_non_proactive:.3f} ms")

print(f"Mean difference (Proactive): {mean_proactive:.3f} ms")


T-statistic: 1.08, P-value: 0.2850
Cohen's d: 0.396
Degrees of Freedom: 38
95% CI for mean difference: [-12.448, 31.922]
Mean difference (Non-Proactive): 30.018 ms
Mean difference (Proactive): 20.281 ms


## Correlate survey scores with SSRT

In [105]:
avg_df = merged_df.merge(survey_scores, on='subject_id', how='left')
avg_df['difference_ai_disengaged_and_non_ai_ssrt'] = avg_df['ai'] - avg_df['non_ai']
correlation, pval = stats.pearsonr(avg_df['average_score'], avg_df["difference_ai_disengaged_and_non_ai_ssrt"])
print(f"Correlation and p-value between SSRT and survey scores: {correlation}, {pval}")

Correlation and p-value between SSRT and survey scores: -0.06668117800291128, 0.6826781257173478


## Violations of Context Independence in Simple Stop Data

### a) Non-AI SSRT vs Simple Stop SSRT Excluding trials with SSD < 200 ms

In [106]:
print(f'Mean Non-AI SSRT: {np.mean(merged_df['non_ai'])}')
print(f'Simple Stop SSRT Excluding Trials with SSD < 200ms: {np.mean(merged_df['simple_stop_ssrt_without_short_ssd_trials'])}')
non_ai_vs_simple = stats.ttest_rel(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
print(f"T-statistic: {non_ai_vs_simple.statistic}, p-value = {non_ai_vs_simple.pvalue}")

cohens_d2 = cohens_d_paired(merged_df['non_ai'], merged_df['simple_stop_ssrt_without_short_ssd_trials'])
mean_diff2, ci2 = calculate_ci_for_difference(merged_df['non_ai'], merged_df['simple_stop_ssrt_without_short_ssd_trials'])
print(f"Cohen's d: {cohens_d2:.3f}")
print(f"Mean difference = {mean_diff2:.3f} ms")
print(f"95% CI: [{ci2[0]:.3f}, {ci2[1]:.3f}] ms")

Mean Non-AI SSRT: 276.16931123585636
Simple Stop SSRT Excluding Trials with SSD < 200ms: 211.18777607787916
T-statistic: 13.16166244544354, p-value = 6.286586264119194e-16
Cohen's d: 3.208
Mean difference = 64.982 ms
95% CI: [55.819, 74.144] ms


### b) Non-AI SSRT vs Simple Stop SSRT Excluding subjects with average SSDs < 200ms

In [108]:
# Remove rows with NaN in simple_stop_ssrt_without_short_ssd_subs
filtered_df = merged_df.dropna(subset=['simple_stop_ssrt_without_short_ssd_subs'])

print(f'Mean Non-AI SSRT: {np.mean(filtered_df['non_ai'])}')
print(f'Simple Stop SSRT Excluding Subjects with SSD < 200ms: {np.mean(filtered_df['simple_stop_ssrt_without_short_ssd_subs'])}')
non_ai_vs_simple = stats.ttest_rel(filtered_df['non_ai'], filtered_df['simple_stop_ssrt_without_short_ssd_subs'])
print(f"T-statistic: {non_ai_vs_simple.statistic}, p-value = {non_ai_vs_simple.pvalue}")

cohens_d2 = cohens_d_paired(filtered_df['non_ai'], filtered_df['simple_stop_ssrt_without_short_ssd_subs'])
mean_diff2, ci2 = calculate_ci_for_difference(filtered_df['non_ai'], filtered_df['simple_stop_ssrt_without_short_ssd_subs'])
print(f"Cohen's d: {cohens_d2:.3f}")
print(f"Mean difference = {mean_diff2:.3f} ms")
print(f"95% CI: [{ci2[0]:.3f}, {ci2[1]:.3f}] ms")

Mean Non-AI SSRT: 279.75175769064026
Simple Stop SSRT Excluding Subjects with SSD < 200ms: 211.80287395613362
T-statistic: 13.167887180372727, p-value = 1.8257545943366015e-14
Cohen's d: 3.242
Mean difference = 67.949 ms
95% CI: [57.438, 78.460] ms
